<a href="https://colab.research.google.com/github/LeoCh97/Tesi/blob/master/CNN_attempt1_with_data_augmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#!rm -r log
#!mkdir log

In [0]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, AveragePooling2D, Activation, Dense, Flatten, Dropout
from tensorflow.keras.utils import to_categorical, normalize
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping
import matplotlib.pyplot as plt
import pickle
import time

earlystopping = EarlyStopping(monitor='val_acc', patience=3, restore_best_weights=True)

pickle_in = open("X_train.pickle","rb")
X_train = pickle.load(pickle_in)

pickle_in = open("y_train.pickle","rb")
y_train = pickle.load(pickle_in)

pickle_in = open("X_val.pickle","rb")
X_val = pickle.load(pickle_in)

pickle_in = open("y_val.pickle","rb")
y_val = pickle.load(pickle_in)

caty_train = to_categorical(y_train, num_classes=33)
caty_val = to_categorical(y_val, num_classes=33)

FileNotFoundError: ignored

In [0]:
fig = plt.figure(figsize=(5,4))
for i in range(3):
    for j in range(3):
        ax = fig.add_subplot(3, 3, i * 3 + j + 1)
        ax.imshow(X_train[i * 3 + j])

In [0]:
X_train = normalize(X_train, axis=2)

fig = plt.figure(figsize=(5,4))
for i in range(3):
    for j in range(3):
        ax = fig.add_subplot(3, 3, i * 3 + j + 1)
        ax.imshow(X_train[i * 3 + j])

In [0]:
imgaug_gen = ImageDataGenerator(
                featurewise_center=True,
                featurewise_std_normalization=True,
                rotation_range=20,
                width_shift_range=0.1,
                height_shift_range=0.1,
                horizontal_flip=True,
                vertical_flip=True)

imgaug_gen.fit(X_train)

In [0]:
dense_layers = [0, 1]
layer_sizes = [64, 128, 256]
conv_layers = [1, 2, 3]

for dense_layer in dense_layers:
    for layer_size in layer_sizes:
        for conv_layer in conv_layers:
            NAME = "{}-conv-{}-nodes-{}-dense-{}".format(conv_layer, layer_size, dense_layer, int(time.time()))
            tensorboard = TensorBoard(log_dir="log/{}".format(NAME))
            print(NAME)

            model = Sequential()

            model.add(Conv2D(layer_size, (5, 5), strides=2, input_shape=X_train.shape[1:]))
            model.add(Activation('relu'))
            model.add(MaxPooling2D(pool_size=(2, 2)))

            for l in range(conv_layer-1):
                model.add(Conv2D(layer_size, (5, 5), strides=2))
                model.add(Activation('relu'))
                model.add(MaxPooling2D(pool_size=(2, 2)))

            model.add(Flatten())
            for _ in range(dense_layer):
                model.add(Dense(layer_size))
                model.add(Activation('relu'))

            model.add(Dense(33))
            model.add(Activation('softmax'))
            
            model.compile(loss="categorical_crossentropy",
                         optimizer="adam",
                         metrics=["accuracy"])
            
            model.fit_generator(imgaug_gen.flow(X_train, caty_train, batch_size=32, save_to_dir="augmented_images"), steps_per_epoch= len(X_train)/32, epochs=10, validation_data=(X_val, caty_val), callbacks=[tensorboard, earlystopping])